# Modular pipelines with scikit-learn

In this notebook you will practice how to make your analyses modular, using `sklearn` tools. You will see how this structure can help build more sophisticated setups.

We will work on a classification task, using a credit risk dataset from Taiwan, with the goal of predicting the risk of credit default.

[Dataset reference](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients).

#### Import libraries

In [1]:
# Hide warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import pickle

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer, PolynomialFeatures
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# For custom transformers
from sklearn.base import TransformerMixin, BaseEstimator

Load the dataset from `data/default.xls`:

In [3]:
# Install xlrd for reading .xls files
%pip install xlrd

In [4]:
data = pd.read_excel("../data/default.xls", skiprows=1)
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


## Basic pipeline

#### Create data matrices

- Create a feature matrix `X` and a target vector `y` from `data`. 
    * The target to be predicted is given in the last column. 
    * The feature matrix should consist of the rest of the columns (except for `'ID'`).
- Create train/test splits with `train_test_split()`, specifying that the data should be stratified by `y`.

In [ ]:
# Your code here...
X = data.drop(columns=["ID","default payment next month"]) # could also do X = data.iloc[:,1:-1]
y = data["default payment next month"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



Lets have a look at the unique values in the target to check whether the dataset is imbalanced:

In [7]:
print("Unique values in y: {}".format(np.unique(y)))
print("Number of 1s in y: {}/{}".format(sum(y), len(y))) # data is imbalanced

Unique values in y: [0 1]
Number of 1s in y: 6636/30000


In [8]:
X_train.shape, X_test.shape

((24000, 23), (6000, 23))

#### Fit model and classify
Let's use `.fit()` and find the score of a Decision Tree classifier:

In [9]:
logreg = DecisionTreeClassifier()
logreg.fit(X_train, y_train)

print("Score: ", logreg.score(X_test, y_test))

y_pred = logreg.predict(X_test)

print(classification_report(y_test, y_pred))

Score:  0.7186666666666667
              precision    recall  f1-score   support

           0       0.83      0.81      0.82      4673
           1       0.37      0.39      0.38      1327

    accuracy                           0.72      6000
   macro avg       0.60      0.60      0.60      6000
weighted avg       0.72      0.72      0.72      6000



#### Understanding the Classification Report

The classification report provides several key metrics for evaluating your model's performance:

**Per-Class Metrics:**
- **Precision**: Of all predictions for this class, what percentage were correct?
  - Class 0 (no default): 83% - when the model predicts "no default", it's correct 83% of the time
  - Class 1 (default): 37% - when it predicts "default", it's only correct 37% of the time
  
- **Recall (Sensitivity)**: Of all actual instances of this class, what percentage did we catch?
  - Class 0: 81% - we correctly identify 81% of actual non-defaulters
  - Class 1: 39% - we only catch 39% of actual defaulters (missing 61%!)
  
- **F1-Score**: Harmonic mean of precision and recall (balances both metrics)
  - Class 0: 0.82 (good performance)
  - Class 1: 0.38 (poor performance)
  
- **Support**: Number of actual instances in each class in the test set
  - 4,673 non-defaulters vs 1,327 defaulters (confirming the imbalance)

**Overall Metrics:**
- **Accuracy (0.72)**: Overall, 72% of predictions are correct - but this is misleading with imbalanced data!
- **Macro avg**: Simple average across classes (treats both equally, regardless of imbalance)
- **Weighted avg**: Average weighted by support (gives more weight to the majority class)

**⚠️ Key Insight for Imbalanced Data:**
The model performs well on the majority class (0) but poorly on the minority class (1 - defaults). This is problematic because in credit risk, **missing a defaulter (39% recall) is often more costly than falsely flagging a non-defaulter**. The high overall accuracy (72%) masks this poor performance on the critical minority class.

#### Scikit-learn pipeline

Now use the pipeline function of scikit-learn to arrive at the same results in three lines of code. Create a simple Decision Tree pipeline with `make_pipeline()` and test its predictions with `cross_val_score()`. Use a single split from now on, by setting `cv` to `ShuffleSplit(n_splits=1)`.

In [24]:
# Your code here...
pipeline = make_pipeline(DecisionTreeClassifier())
split = ShuffleSplit(n_splits=1, random_state=42) # define a single split. If we have multiple splits, we need to aggregate the results
cross_val_score(pipeline, X, y, cv=split)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.81      0.82      4673
           1       0.37      0.39      0.38      1327

    accuracy                           0.72      6000
   macro avg       0.60      0.60      0.60      6000
weighted avg       0.72      0.72      0.72      6000



### Preprocessing pipeline

Our initial analysis overlooked several aspects that may hinder the performance of a predictive model. One is that the features should be transformed to an appropriate input type for a linear model. Here are the features we want to build:
* `SEX`, `EDUCATION`, `MARRIAGE`: One-hot encoded features for categorical columns.
* `PAY_...`, `BILL_AMT...`, `PAY_AMT...`: these features correspond to bills and payments made in the previous months. They're highly correlated so we will first scale them and then apply PCA.
* `AGE`, `LIMIT_BAL`: these are numerical values. We will use `FeatureUnion` to create log and polynomial versions of them.
* `Unpaid_by_user` / `avg_unpaid`: the ratio amount unpaid by the user divided by the average unpaid bill last month for all users. This should give us an idea of how bad at paying bills the user is.

We'll go through each of them step-by-step, leveraging scikit-learn transformers.

#### Column one-hot transformation

We will create one-hot encoded features for the categorical columns `SEX`, `EDUCATION`, `MARRIAGE`. The standard process would be to create the new columns and add them to a new data matrix, and for each such preprocessing step we would have to create a new data set variation. Instead, we can create a transformation to be used in a pipeline.

Create a one-hot transformer for the categorical variables, using `make_column_transformer()` and the `OneHotEncoder()`. The `ColumnTransformer` allows you to apply different transformations in parallel to different columns. Remember to set `remainder='passthrough'` to retain all other columns. 

Create a pipeline with the one-hot preprocessor and a Decision Tree. Test the model predictions with `cross_val_score()`.


In [ ]:
# Your code here...
cat_cols = ["SEX", "EDUCATION", "MARRIAGE"]

one_hot_processor = make_column_transformer(
    (OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first'), cat_cols),
    remainder='passthrough')

pipeline = make_pipeline(one_hot_processor, DecisionTreeClassifier())
cross_val_score(pipeline, X, y, cv=split) # cv=split means we use the same split as before

print(classification_report(y_test, y_pred)) # Note that report is same as before because one-hot encoding doesnt make a big difference with decision trees as they can handle categorical variables inherently

              precision    recall  f1-score   support

           0       0.83      0.81      0.82      4673
           1       0.37      0.39      0.38      1327

    accuracy                           0.72      6000
   macro avg       0.60      0.60      0.60      6000
weighted avg       0.72      0.72      0.72      6000



#### Understanding the One-Hot Encoding Implementation

**Step-by-step breakdown:**

1. **Define categorical columns:**
   ```python
   cat_cols = ["SEX", "EDUCATION", "MARRIAGE"]
   ```
   These are the columns containing categorical data that need encoding.

2. **Create the ColumnTransformer:**
   ```python
   one_hot_processor = make_column_transformer(
       (OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first'), cat_cols),
       remainder='passthrough')
   ```
   
   **OneHotEncoder parameters:**
   - `sparse_output=False`: Returns a dense array (regular numpy array) instead of sparse matrix - easier to work with
   - `handle_unknown='ignore'`: If new categories appear in test/production data, ignore them gracefully (critical for deployment!)
   - `drop='first'`: Drops first category to avoid multicollinearity (the "dummy variable trap")
   
   **ColumnTransformer behavior:**
   - Applies `OneHotEncoder` to `SEX`, `EDUCATION`, `MARRIAGE`
   - `remainder='passthrough'`: Keeps all other columns unchanged (AGE, LIMIT_BAL, payment history, etc.)

3. **Create pipeline:**
   ```python
   pipeline = make_pipeline(one_hot_processor, DecisionTreeClassifier())
   ```
   Chains preprocessing with modeling in a single object.

4. **Evaluate with cross-validation:**
   ```python
   cross_val_score(pipeline, X, y, cv=split)
   ```
   Tests the complete pipeline (preprocessing + model) on the data.

**Example transformation:**
- **Before:** `SEX=1` (male), `SEX=2` (female)
- **After (with drop='first'):** Creates `SEX_2` column → 1 if female, 0 if male
- The first category (male) becomes the reference baseline

**Why pipelines?**
✅ Prevents data leakage (fit only on training data in each fold)
✅ Makes code cleaner and more maintainable
✅ Easier deployment (preprocessing + model in one object)

#### Reducing correlated features with Scaler and PCA

We will scale the payment columns and reduce their dimensionality with PCA.

Create a sequential transformer that first scales the data and then applies PCA.

Choose a sensible value for `n_components`, which sets the number of components to keep (note that `n_components` can be specified as a `float` saying what proportion of variance is retained).

In [ ]:
# Your code here...
# Create a pipeline that scales then applies PCA
# Keep 80% of variance (or use a specific number like n_components=5)
payment_transformer = make_pipeline(
    MinMaxScaler(), # is a safer scaling method that scales features to a range between 0 and 1. Default is StandardScaler which standardizes to mean 0 and variance 1
    PCA(n_components=.8) # result is to get enough components to explain 80% of variance. 80% is a good rule of thumb
)

Great, let's keep this transformer for later when we'll group all the pieces together.

#### Adding handcrafted features with `FeatureUnion`

`FeatureUnion` can be applied on the data to add features (for example: interaction features or transformed features). In this dataset we want to create new features by combining and transforming `AGE` and `LIMIT_BAL`.

Create a transformer that adds the log features with `FunctionTransformer()`, and a polynomial pairwise transformation (check the `PolynomialFeatures` documentation). For the latter transformer, specify `include_bias=False` and `interaction_only=True`.

Test the transformation by calling `.fit_transform()` on `X_train`, columns `AGE` and `LIMIT_BAL`. How many new columns does this transformation create?

In [25]:
# Your code here...
numerical_transformer = FeatureUnion([
    ('log', FunctionTransformer(np.log)), # apply log transformation
    ('polynomial', PolynomialFeatures(2, include_bias=False, # apply polynomial features transformation
    interaction_only=True)) # e.g., AGE^2, AGE*LIMIT_BAL, LIMIT_BAL^2
])

numerical_transformer.fit_transform(X_train[["AGE", "LIMIT_BAL"]]).shape


(24000, 5)

#### Understanding the FeatureUnion and Numerical Transformer

**What is FeatureUnion?**

`FeatureUnion` combines multiple transformers in **parallel** (horizontally stacking features), unlike `Pipeline` which chains them sequentially.

**Breaking down the code:**

1. **`FunctionTransformer(np.log)`**:
   - Applies logarithmic transformation to both `AGE` and `LIMIT_BAL`
   - Creates 2 new features: `log(AGE)` and `log(LIMIT_BAL)`
   - Why? Log transforms can help handle skewed distributions and reduce the effect of outliers

2. **`PolynomialFeatures(2, include_bias=False, interaction_only=True)`**:
   - `degree=2`: Considers up to 2nd degree polynomial features
   - `include_bias=False`: Doesn't add a constant column (bias term)
   - `interaction_only=True`: **Only creates the interaction term** (`AGE × LIMIT_BAL`), not squared terms
   - Creates 3 features: `AGE`, `LIMIT_BAL`, `AGE × LIMIT_BAL`

**Total features created: 5**
- From log transformer: 2 features (log_AGE, log_LIMIT_BAL)
- From polynomial transformer: 3 features (AGE, LIMIT_BAL, AGE×LIMIT_BAL)
- These 5 features are concatenated horizontally (side-by-side)

**Important Note:** The comment in the code saying "e.g., AGE^2, AGE*LIMIT_BAL, LIMIT_BAL^2" is **misleading**! With `interaction_only=True`, you **don't** get squared terms (AGE², LIMIT_BAL²). You only get the original features plus their interaction.

**Why use FeatureUnion?**
- Captures different aspects of the data (linear, logarithmic, and interaction effects)
- Enriches feature space for the model to learn from
- Allows the model to discover non-linear relationships

Note that if you have two features `a` and `b`, then `PolynomialFeatures(2, include_bias=False, interaction_only=True)` includes features `a`, `b` and `a*b`. If you remove `interaction_only`, it would include `a`, `b`, `a*b`, `a**2`, `b**2`. 

#### Custom Transformer

We'll create a custom transformer that indicates if a user is above or below average for the unpaid bill last month: `unpaid_by_user > avg_unpaid`.

For this we need to memorise the average amount unpaid by all users in the `.fit()` step, and then apply the transformation for any user in the `.transform()` step.

We've defined the template for the class you need to implement below. Note that it extends `TransformerMixin` and `BaseEstimator`; that's required to define a new `Transformer` in the right format for `sklearn`.

Follow the instructions below. You can run the code in the next cell to test your implementation.

1. Implement the `.fit()` method:
  * This method takes `X` and `y` (`y` is optional). 
  * It needs to compute `avg_unpaid_6`, `avg_unpaid_5`, and `avg_unpaid_4` - the amount unpaid on average for all users in the given training set for month 4, 5, 6. For example `avg_unpaid_6` is the mean of `BILL_AMT6` - `PAY_AMT6`.
  * To be compatible with other `sklearn` tools, `.fit()` needs to return `self`.


2. Implement the `.transform()` method:
  * This method takes `X` as input and needs to return a new DataFrame with columns `Unpaid_ratio_6`, `Unpaid_ratio_5`, and `Unpaid_ratio_4`, containing the same number of rows as `X`.
  * Each value indicates if the amount unpaid by user (`BILL_AMT` - `PAY_AMT`) is higher than the average amount saved in `.fit()`.

In [ ]:
class UnpaidTransformer(TransformerMixin, BaseEstimator):
    
    def fit(self, X, y=None):
        # Your code here...
        self.avg_unpaid_6 = (X["BILL_AMT6"] - X["PAY_AMT6"]).mean()
        self.avg_unpaid_5 = (X["BILL_AMT5"] - X["PAY_AMT5"]).mean()
        self.avg_unpaid_4 = (X["BILL_AMT4"] - X["PAY_AMT4"]).mean()
        
        return self
        pass # --- IGNORE ---

    def transform(self, X, y=None):
        # Your code here...
        X_6 = (X["BILL_AMT6"] - X["PAY_AMT6"]) > self.avg_unpaid_6
        X_5 = (X["BILL_AMT5"] - X["PAY_AMT5"]) > self.avg_unpaid_5
        X_4 = (X["BILL_AMT4"] - X["PAY_AMT4"]) > self.avg_unpaid_4

        X_new = pd.DataFrame({"Unpaid_ratio_6": X_6, "Unpaid_ratio_5": X_5, "Unpaid_ratio_4": X_4})
        return X_new # New dataFrame with the new features
        pass # --- IGNORE ---

In [27]:
# Instantiate the transformer
unpaid_transfomer = UnpaidTransformer()

# Test .fit()
unpaid_transfomer.fit(X_train)
print(unpaid_transfomer.avg_unpaid_6)

# Test .transform()
unpaid_transfomer.transform(X_train).head()

33446.074375


,Unpaid_ratio_6,Unpaid_ratio_5,Unpaid_ratio_4
22788,True,True,True
29006,False,False,False
16950,False,False,False
22280,True,True,True
11346,False,False,False


#### Build a joint preprocessor

Now that we have all the transformers we need, we can create a single object using `ColumnTransformer`. First we need to define a list of columns that will be processed separately:

In [28]:
payment_cols = [col for col in X_train.columns if col.startswith("PAY_") or col.startswith("BILL_")]

num_cols = ["AGE", "LIMIT_BAL"]

# Features needed to compute the unpaid features:
unpaid_cols = ["PAY_AMT4", "PAY_AMT5", "PAY_AMT6", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"]
cat_cols = ["SEX", "EDUCATION", "MARRIAGE"]

Now let's create a `ColumnTransfomer` object, which takes a list of tuples as input. Each tuple needs to have the following format: (`name`, `transformer`, `columns`).

In [29]:
# Your code here...
preprocessor = ColumnTransformer([("payments", payment_transformer, payment_cols), 
                                  ("numerical", numerical_transformer, num_cols),
                                  ("unpaid", unpaid_transfomer, unpaid_cols),
                                  ("categorical", OneHotEncoder(), cat_cols)])


Now we have a preprocessor object that we can reuse for any new data. The next step is to create a predictive model with it.

## Optimizing models

In order to evaluate different predictive algorithms, we can create several pipeline objects, each combining our preprocessor together with a different algorithm. 

Create new pipeline objects (`dtc` and `rfc`) with our joint preprocessor for the Decision Tree and the Random Forest classifiers. Test them with `cross_val_score()`.

In [ ]:
# Your code here...
dtc = make_pipeline(preprocessor, DecisionTreeClassifier())
print(cross_val_score(dtc,X,y,cv=split))

rfc = make_pipeline(preprocessor, RandomForestClassifier()) # better performance than decision tree as it is an ensemble method
print(cross_val_score(rfc,X,y,cv=split))

[0.72266667]
[0.805]
[0.805]


#### Using grid search with a pipeline

One major advantage of using a pipeline is that now we have both the processing **and** predictive modeling in one object, which can be trained and optimized jointly. It means that on every fold it is trained on, we will refit the model and also the transformations, which ensures we do not have data leakage and makes the evaluation more accurate. 

When tuning our model we can tune the hyperparameters of the model, and also some processing parameters, such as the number of PCA components. This would be much harder with a regular sequential preprocessing-then-training scenario.

Below we use grid search to tune the `max_depth` and `min_samples_split` of the Random Forest model, together with the `n_components` in PCA.

Note: to refer to a parameter, you need to use the name of the step, followed by `__`, then the parameter name. Since the PCA model is deep inside our pipeline, the name is a bit more complicated. Using `.named_steps` allows us to see what's inside our pipeline model:

In [32]:
rfc.named_steps # view the steps in the pipeline

{'columntransformer': ColumnTransformer(transformers=[('payments',
                                  Pipeline(steps=[('minmaxscaler',
                                                   MinMaxScaler()),
                                                  ('pca',
                                                   PCA(n_components=0.8))]),
                                  ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
                                   'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
                                   'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5',
                                   'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2',
                                   'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5',
                                   'PAY_AMT6']),
                                 ('numerical',
                                  FeatureUnion(transformer_list=[('log',
                                                                  FunctionTransformer(func=<ufunc 'log'>)),
                   

Here PCA is in `columntransformer`, then `payments` (the Pipeline), then `pca` (one can use `.get_params()` to make sure we got the name of the parameter right). 

Use `GridSearchCV()` to find the parameters with the best accuracy. Print these parameters and the corresponding score, with `.best_params_` and `.best_score_`.

In [ ]:
# Your code here...
params = {'columntransformer__payments__pca__n_components': [0.5, 0.98], # __ to access nested parameters in the pipeline. Travel from outside to inside
          'randomforestclassifier__max_depth': [5, 30],
          'randomforestclassifier__min_samples_split': [2, 5]}

gcv = GridSearchCV(rfc, param_grid=params, n_jobs=-1, cv=split)
gcv.fit(X_train, y_train)

print(gcv.best_params_)
print(gcv.best_score_) # score is a minor improvement over previous score

{'columntransformer__payments__pca__n_components': 0.98, 'randomforestclassifier__max_depth': 5, 'randomforestclassifier__min_samples_split': 2}
0.81125


#### Recover and fit the model with the best hyper-parameters

You can recover the best hyper-parameters of the grid-search with `.set_params()`.

In [34]:
# Your code here...
rfc.set_params(**gcv.best_params_); # ** unpacks the dictionary of best params
rfc.fit(X_train, y_train);

### Saving pre-trained models

Now save the best pre-trained model to be used later, using the `pickle` library. Write your model to `"data/rf_model.pickle"`.

In [36]:
# Your code here...
with open("../data/rf_model.pickle", "wb") as f:
    pickle.dump(rfc, f)

Now load your model and print its score again, to check that you have successfully saved it.

In [ ]:
with open("../data/rf_model.pickle", "rb") as f:
    rfc_pickle = pickle.load(f)
cross_val_score(rfc_pickle, X, y, cv=split) # results are variable because of random shuffling

array([0.80733333])

Pickle is really useful to save trained models, but keep in mind that it does not save dependencies. To be able to load your model, you need to import the code it relies on: external libraries and our own class definitions. Because of this, it is common to get errors when trying to load a model trained with an old version of a library.

#### Reproducible experiments

To make sure your model was not just lucky to get good results for a given training session, it is important to know how to make reproducible experiments. Apart from having the same code and data of course, to be reproducible, one needs to have same random number generator state. 

Run three instances of training your pipeline and testing it with `cross_val_score()`. Before two of them set the random seed to `1234`, with `np.random.seed()`. Do the results match exactly?

In [39]:
# Your code here...
np.random.seed(); 
rfc1 = make_pipeline(preprocessor, RandomForestClassifier()) 
cross_val_score(rfc1, X, y, cv=split)

array([0.81966667])

In [41]:
# Your code here...
np.random.seed(1234)
rfc2 = make_pipeline(preprocessor, RandomForestClassifier()) 
cross_val_score(rfc2, X, y, cv=split)

array([0.813])

In [ ]:
# Your code here...
np.random.seed(1234)
rfc3 = make_pipeline(preprocessor, RandomForestClassifier()) 
cross_val_score(rfc3, X, y, cv=split) # same results as rfc2 because of same random seed

array([0.813])